In [22]:
#x^2+5x-4-x^3-5x^2+4x-13
class Operation():                  
    
    def __init__(self, input_nodes = []):
        self.input_nodes = input_nodes 
        self.output_nodes = []
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
        
    def compute(self):
        pass

In [23]:
class add(Operation):
    
    def __init__(self, x, y):
         
        super().__init__([x, y])

    def compute(self, x_var, y_var):
         
        self.inputs = [x_var, y_var]
        return x_var + y_var
    
class multiply(Operation):
     
    def __init__(self, a, b):
        
        super().__init__([a, b])
    
    def compute(self, a_var, b_var):
         
        self.inputs = [a_var, b_var]
        return a_var * b_var
    
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        
        _default_graph.placeholders.append(self)
        
class Variable():
    
    def __init__(self, initial_value = None):
        
        self.value = initial_value
        self.output_nodes = []
        
         
        _default_graph.variables.append(self)

In [24]:
class Graph():
    
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        global _default_graph
        _default_graph = self

In [28]:
g = Graph()
g.set_as_default()

In [29]:
x = Placeholder()

In [30]:
a = Variable(5)
b = Variable(-4)
c = Variable(-5)
d = Variable(4)
e = Variable(-13)
m = Variable(-1)

In [31]:
f = multiply(x,x) #x^2
g = multiply(c,f) #-5x^2
h = multiply(f,x) #x^3
z = multiply(h,m) #-x^3
i = multiply(b,x) #4x
j = multiply(a,x) #5x

k = add(f,j) # x^2+5x
l = add(k,b) # x^2+5x-4
s = add(l,z) # x^2+5x-4-x^3
n = add(s,g) # x^2+5x-4-x^3-5x^2
o = add(n,i) # x^2+5x-4-x^3-5x^2+4x
p = add(o,e) # x^2+5x-4-x^3-5x^2+4x-13


In [32]:
import numpy as np

def traverse_postorder(operation):
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [33]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:
            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                node.output = node.value
                
            else:                
                node.inputs = [input_node.output for input_node in node.input_nodes]
                 
                node.output = node.compute(*node.inputs)
                
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        return operation.output

In [34]:
sess = Session()

In [35]:
res = sess.run(operation=p, feed_dict = {x:[7, 2, 10]})

In [36]:
res


array([ -549,   -39, -1407])

In [ ]:
z